# Competition

In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold, train_test_split
import optuna
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

## Preprocessing

### Preprocessing Utilities

In [8]:
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [9]:
def log_return(wap):
    return np.log(wap).diff()

In [10]:
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [11]:
def count_unique(series):
    return len(np.unique(series))

In [12]:
# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv(config.data_dir + 'train.csv')
    test = pd.read_csv(config.data_dir + 'test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

In [81]:
class Config:
    seed = 123
    data_dir = '/data/'
    time_gap = 100
    
    feature_dict_book = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    feature_dict_trade = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    model_params = {
        "xgb_bl": {
            "objective": "reg:squarederror",
            "booster": "gblinear",
            "nthread": -1,
            "eta": 0.3,
            "max_depth": 8,
            "min_child_weight": 1,
            "sampling_method": "gradient_based",
            "tree_method": "gpu_hist"  # turn it on for GPU
        },
        "xgb_tuning": {
            "objective": "reg:squarederror",
            "booster": "gbtree",
            "nthread": -1,
            "tree_method": "gpu_hist",
            'max_depth': 7,
            'eta': 0.03,
            'lambda': 0.01,
            "subsample": 0.2,
            "colsample_bytree": 0.33,
            "sampling_method": "uniform"
        },
        "xgb_optuna": {
            "objective": "reg:squarederror",
            "booster": "gblinear",
            "nthread": -1,
            "tree_method": "gpu_hist",
            'max_depth': 7,
            'eta': 0.03,
            'lambda': 1.0979256871605507e-06,
            'gamma': 2.3321112461277414e-08,
            'alpha': 0.006405029944559645,
            "sampling_method": "gradient_based"
        }
    }

In [82]:
config = Config

### Preprocessing of Book

In [36]:
from functools import reduce

def book_preprocessor(file_path):
    # Function to preprocess book data (for each stock id)
    
    df = pd.read_parquet(file_path)
    
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Function to get group stats for different windows (seconds in bucket)
        
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(config.feature_dict_book).reset_index()
        
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
            df_feature.rename({f'time_id__{seconds_in_bucket}': 'time_id_'}, axis=1, inplace=True)
        return df_feature
    
    # Get the stats for different windows
    df_list = [get_stats_window(seconds_in_bucket = 0, add_suffix = False)]
    
    time_slices = [t * config.time_gap for t in range(1, 600 // config.time_gap)]
    for t in time_slices:
        df_list += [get_stats_window(seconds_in_bucket = t, add_suffix = True)]
    
    df_feature = reduce(lambda left, right: pd.merge(left, right, on='time_id_'), df_list)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature.loc[:, 'row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    
    return df_feature

### Preprocessing of Trade

In [37]:
def trade_preprocessor(file_path):
    # Function to preprocess trade data (for each stock id)
    
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Function to get group stats for different windows (seconds in bucket)
        
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(config.feature_dict_trade).reset_index()
        
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
            df_feature.rename({f'time_id__{seconds_in_bucket}': 'time_id_'}, axis=1, inplace=True)
        return df_feature
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    def process_trade_features(df):
        lis = []
        for n_time_id in df['time_id'].unique():
            df_id = df[df['time_id'] == n_time_id]        
            tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
            f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
            f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
            df_max =  np.sum(np.diff(df_id['price'].values) > 0)
            df_min =  np.sum(np.diff(df_id['price'].values) < 0)
            abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
            energy = np.mean(df_id['price'].values**2)
            iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
            abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
            energy_v = np.sum(df_id['size'].values**2)
            iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)

            lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                       'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})

        df_lr = pd.DataFrame(lis)
        return df_lr
    
    # Get the stats for different windows
    df_list = [get_stats_window(seconds_in_bucket = 0, add_suffix = False)]
    
    time_slices = [t * 100 for t in range(1, 600 // config.time_gap)]
    for t in time_slices:
        df_list += [get_stats_window(seconds_in_bucket = t, add_suffix = True)]
        
    df_feature = reduce(lambda left, right: pd.merge(left, right, on='time_id_'), df_list)
    df_lr = process_trade_features(df)
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    
    return df_feature

### Preprocess of Time Stock

In [38]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = [col for col in df.columns if 'realized_volatility' in col]

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

### Overall Preprocessing

In [39]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = config.data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = config.data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = config.data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = config.data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

## Traning

### Utilities

In [40]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, xgb_train):
    y_true = xgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred)

### Modeling

In [83]:
def train_and_evaluate(train, test):
    params = config.model_params['xgb_tuning']
    
    # Split features and target
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    x_test = test.drop(['row_id', 'time_id'], axis = 1)
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    x_test['stock_id'] = x_test['stock_id'].astype(int)
    dtest = xgb.DMatrix(x_test)
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = config.seed, shuffle = True)
    # Iterate through each fold
    for fold, (train_idx, val_idx) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
        x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

        x_train["stock_id"] = x_train["stock_id"].astype(int)
        x_val["stock_id"] = x_val["stock_id"].astype(int)

        dtrain = xgb.DMatrix(x_train, label=y_train, weight=1/np.square(y_train), enable_categorical=True)
        dval = xgb.DMatrix(x_val, label=y_val, weight=1/np.square(y_val), enable_categorical=True)

        model = xgb.train(params,
                          dtrain=dtrain,
                          evals=[(dtrain, "dtrain"), (dval, "dval")],
                          verbose_eval=50,
                          early_stopping_rounds=100,
                          num_boost_round=1000,
                          feval=feval_rmspe)

        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / 5
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

### Running

In [42]:
# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  5.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished
/home/lhk/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value
/tmp/ipykernel_4300/2304823975.py:2: RuntimeWarning: invalid value encountered in sqrt
/tmp/ipykernel_4300/2304823975.py:2: RuntimeWarning: invalid value encountered in sqrt
/tmp/ipykernel_4300/2304823975.py:2: RuntimeWarning: invalid value encountered in sqrt
/tmp

### Optimization

In [20]:
def objective(trial, train=train):
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.25, random_state=config.seed)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)
    
    param = {
        "objective": "reg:squarederror",
        "booster": "gbtree",
        "max_depth": trial.suggest_int("max_depth", 1, 9),
        "eta": trial.suggest_categorical("eta", [.3, .1]),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "tree_method": "gpu_hist"  # turn it on for GPU
    }

    # Add a callback for pruning.
    model = xgb.train(param,
                      dtrain,
                      evals=[(dtrain, "dtrain"), (dvalid, "dval")],
                      verbose_eval=50,
                      feval=feval_rmspe,
                      num_boost_round=1000,
                      early_stopping_rounds=100)
    y_pred = model.predict(dvalid)
    return rmspe(valid_y, y_pred)

In [21]:
# study = optuna.create_study(
#     pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="minimize"
# )
# study.optimize(objective, n_trials=100)
# print(study.best_trial)

[I 2021-08-29 20:34:39,354] A new study created in memory with name: no-name-44bc5c6c-abce-4390-9535-20c63448ba86


[0]	dtrain-rmse:0.44652	dtrain-RMSPE:207.72995	dval-rmse:0.44651	dval-RMSPE:207.10060
[50]	dtrain-rmse:0.00261	dtrain-RMSPE:1.31693	dval-rmse:0.00261	dval-RMSPE:1.30587
[100]	dtrain-rmse:0.00112	dtrain-RMSPE:0.28281	dval-rmse:0.00115	dval-RMSPE:0.27382
[150]	dtrain-rmse:0.00112	dtrain-RMSPE:0.27976	dval-rmse:0.00115	dval-RMSPE:0.27082
[200]	dtrain-rmse:0.00112	dtrain-RMSPE:0.27975	dval-rmse:0.00115	dval-RMSPE:0.27081
[250]	dtrain-rmse:0.00112	dtrain-RMSPE:0.27975	dval-rmse:0.00115	dval-RMSPE:0.27081
[283]	dtrain-rmse:0.00112	dtrain-RMSPE:0.27975	dval-rmse:0.00115	dval-RMSPE:0.27081


[I 2021-08-29 20:34:48,831] Trial 0 finished with value: 0.2708351141333529 and parameters: {'max_depth': 9, 'eta': 0.1, 'lambda': 0.00021752702272772565, 'gamma': 0.00016957320889323264, 'alpha': 0.6727536315726015}. Best is trial 0 with value: 0.2708351141333529.


[0]	dtrain-rmse:0.34730	dtrain-RMSPE:161.69887	dval-rmse:0.34729	dval-RMSPE:161.20781
[50]	dtrain-rmse:0.00177	dtrain-RMSPE:0.67212	dval-rmse:0.00179	dval-RMSPE:0.66360
[100]	dtrain-rmse:0.00177	dtrain-RMSPE:0.67212	dval-rmse:0.00179	dval-RMSPE:0.66360
[150]	dtrain-rmse:0.00177	dtrain-RMSPE:0.67212	dval-rmse:0.00179	dval-RMSPE:0.66360
[154]	dtrain-rmse:0.00177	dtrain-RMSPE:0.67212	dval-rmse:0.00179	dval-RMSPE:0.66360


[I 2021-08-29 20:34:54,775] Trial 1 finished with value: 0.6635729506493484 and parameters: {'max_depth': 5, 'eta': 0.3, 'lambda': 8.718646811776916e-07, 'gamma': 0.21461108591694744, 'alpha': 0.8331889468812087}. Best is trial 0 with value: 0.2708351141333529.


...

[0]	dtrain-rmse:0.44652	dtrain-RMSPE:207.69064	dval-rmse:0.44651	dval-RMSPE:207.06130
[50]	dtrain-rmse:0.00250	dtrain-RMSPE:1.19772	dval-rmse:0.00254	dval-RMSPE:1.18837
[100]	dtrain-rmse:0.00084	dtrain-RMSPE:0.23640	dval-rmse:0.00105	dval-RMSPE:0.23797
[150]	dtrain-rmse:0.00077	dtrain-RMSPE:0.22060	dval-rmse:0.00103	dval-RMSPE:0.23012
[200]	dtrain-rmse:0.00071	dtrain-RMSPE:0.21010	dval-rmse:0.00102	dval-RMSPE:0.22664
[250]	dtrain-rmse:0.00067	dtrain-RMSPE:0.20136	dval-rmse:0.00102	dval-RMSPE:0.22315
[300]	dtrain-rmse:0.00063	dtrain-RMSPE:0.19347	dval-rmse:0.00101	dval-RMSPE:0.22103
[350]	dtrain-rmse:0.00060	dtrain-RMSPE:0.18762	dval-rmse:0.00101	dval-RMSPE:0.22034
[400]	dtrain-rmse:0.00058	dtrain-RMSPE:0.18164	dval-rmse:0.00101	dval-RMSPE:0.21962
[450]	dtrain-rmse:0.00055	dtrain-RMSPE:0.17725	dval-rmse:0.00101	dval-RMSPE:0.21934
[500]	dtrain-rmse:0.00055	dtrain-RMSPE:0.17548	dval-rmse:0.00101	dval-RMSPE:0.21903
[550]	dtrain-rmse:0.00055	dtrain-RMSPE:0.17548	dval-rmse:0.00101	dval-RMSPE

[I 2021-08-29 21:26:52,179] Trial 98 finished with value: 0.21902760380135355 and parameters: {'max_depth': 8, 'eta': 0.1, 'lambda': 7.731160815642305e-05, 'gamma': 1.1051884632092938e-08, 'alpha': 0.002889281086259594}. Best is trial 91 with value: 0.21680341345875656.


[0]	dtrain-rmse:0.44652	dtrain-RMSPE:207.69076	dval-rmse:0.44651	dval-RMSPE:207.06143
[50]	dtrain-rmse:0.00252	dtrain-RMSPE:1.20386	dval-rmse:0.00254	dval-RMSPE:1.19365
[100]	dtrain-rmse:0.00092	dtrain-RMSPE:0.24736	dval-rmse:0.00106	dval-RMSPE:0.24325
[150]	dtrain-rmse:0.00085	dtrain-RMSPE:0.23431	dval-rmse:0.00104	dval-RMSPE:0.23432
[200]	dtrain-rmse:0.00080	dtrain-RMSPE:0.22489	dval-rmse:0.00103	dval-RMSPE:0.22928
[250]	dtrain-rmse:0.00076	dtrain-RMSPE:0.21730	dval-rmse:0.00102	dval-RMSPE:0.22595
[300]	dtrain-rmse:0.00073	dtrain-RMSPE:0.21073	dval-rmse:0.00101	dval-RMSPE:0.22380
[350]	dtrain-rmse:0.00070	dtrain-RMSPE:0.20606	dval-rmse:0.00101	dval-RMSPE:0.22197
[400]	dtrain-rmse:0.00068	dtrain-RMSPE:0.20151	dval-rmse:0.00101	dval-RMSPE:0.22061
[450]	dtrain-rmse:0.00066	dtrain-RMSPE:0.19751	dval-rmse:0.00100	dval-RMSPE:0.21959
[500]	dtrain-rmse:0.00064	dtrain-RMSPE:0.19348	dval-rmse:0.00100	dval-RMSPE:0.21862
[550]	dtrain-rmse:0.00062	dtrain-RMSPE:0.18961	dval-rmse:0.00100	dval-RMSPE

[I 2021-08-29 21:27:42,251] Trial 99 finished with value: 0.21741674715066586 and parameters: {'max_depth': 7, 'eta': 0.1, 'lambda': 0.00010540741262364088, 'gamma': 1.7742718872263023e-08, 'alpha': 0.005286265405552988}. Best is trial 91 with value: 0.21680341345875656.


FrozenTrial(number=91, values=[0.21680341345875656], datetime_start=datetime.datetime(2021, 8, 29, 21, 19, 57, 344339), datetime_complete=datetime.datetime(2021, 8, 29, 21, 20, 49, 55779), params={'max_depth': 7, 'eta': 0.1, 'lambda': 1.0979256871605507e-06, 'gamma': 2.3321112461277414e-08, 'alpha': 0.006405029944559645}, distributions={'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'eta': CategoricalDistribution(choices=(0.3, 0.1)), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=91, state=TrialState.COMPLETE, value=None)


In [24]:
print(study.best_trial)

FrozenTrial(number=91, values=[0.21680341345875656], datetime_start=datetime.datetime(2021, 8, 29, 21, 19, 57, 344339), datetime_complete=datetime.datetime(2021, 8, 29, 21, 20, 49, 55779), params={'max_depth': 7, 'eta': 0.1, 'lambda': 1.0979256871605507e-06, 'gamma': 2.3321112461277414e-08, 'alpha': 0.006405029944559645}, distributions={'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'eta': CategoricalDistribution(choices=(0.3, 0.1)), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=91, state=TrialState.COMPLETE, value=None)


In [25]:
study.best_trial.params

{'max_depth': 7,
 'eta': 0.1,
 'lambda': 1.0979256871605507e-06,
 'gamma': 2.3321112461277414e-08,
 'alpha': 0.006405029944559645}

#### Exp1 - Optuna Best Param

In [69]:
# Traing and evaluate
test_predictions = train_and_evaluate(train, test)
# Save test predictions
test['target'] = test_predictions
test[['row_id', 'target']].to_csv('submission.csv',index = False)

Training fold 1
[23:38:30] WARNING: ../src/learner.cc:541: 
Parameters: { normalize_type, rate_drop, sample_type, skip_drop } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	dtrain-rmse:0.34874	dtrain-RMSPE:161.45145	dval-rmse:0.34874	dval-RMSPE:161.20386
[10]	dtrain-rmse:0.00986	dtrain-RMSPE:4.56510	dval-rmse:0.00988	dval-RMSPE:4.56506
[20]	dtrain-rmse:0.00049	dtrain-RMSPE:0.22687	dval-rmse:0.00055	dval-RMSPE:0.25314
[30]	dtrain-rmse:0.00038	dtrain-RMSPE:0.17698	dval-rmse:0.00046	dval-RMSPE:0.21114
[40]	dtrain-rmse:0.00036	dtrain-RMSPE:0.16778	dval-rmse:0.00045	dval-RMSPE:0.20796
[50]	dtrain-rmse:0.00035	dtrain-RMSPE:0.16090	dval-rmse:0.00045	dval-RMSPE:0.20835
Training fold 2
[23:38:45] WARNING: ../src/learner.cc:541: 
Parameters: { normalize_type, rate_drop, sample_type, 

#### Exp2 - Baseline

In [84]:
# Traing and evaluate
test_predictions = train_and_evaluate(train, test)
# Save test predictions
submit_df = test.copy()
submit_df['target'] = test_predictions
submit_df[['row_id', 'target']].to_csv('submission.csv',index = False)

Training fold 1
[0]	dtrain-rmse:0.48326	dtrain-RMSPE:223.81732	dval-rmse:0.48325	dval-RMSPE:223.01353
[50]	dtrain-rmse:0.10539	dtrain-RMSPE:48.81221	dval-rmse:0.10539	dval-RMSPE:48.63499
[100]	dtrain-rmse:0.02300	dtrain-RMSPE:10.65103	dval-rmse:0.02300	dval-RMSPE:10.61182
[150]	dtrain-rmse:0.00504	dtrain-RMSPE:2.33579	dval-rmse:0.00505	dval-RMSPE:2.32836
[200]	dtrain-rmse:0.00120	dtrain-RMSPE:0.55351	dval-rmse:0.00121	dval-RMSPE:0.55651
[250]	dtrain-rmse:0.00052	dtrain-RMSPE:0.23860	dval-rmse:0.00054	dval-RMSPE:0.25013
[300]	dtrain-rmse:0.00045	dtrain-RMSPE:0.20862	dval-rmse:0.00048	dval-RMSPE:0.22321
[350]	dtrain-rmse:0.00044	dtrain-RMSPE:0.20489	dval-rmse:0.00048	dval-RMSPE:0.22206
[400]	dtrain-rmse:0.00044	dtrain-RMSPE:0.20252	dval-rmse:0.00048	dval-RMSPE:0.22093
[450]	dtrain-rmse:0.00043	dtrain-RMSPE:0.20118	dval-rmse:0.00048	dval-RMSPE:0.21986
[500]	dtrain-rmse:0.00043	dtrain-RMSPE:0.19985	dval-rmse:0.00048	dval-RMSPE:0.22028
[550]	dtrain-rmse:0.00043	dtrain-RMSPE:0.19876	dval-rms